In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [2]:
with open('data/get_movies.pkl', 'rb') as f:
    t = pickle.load(f)
    
movie_dic = t[0]
movie_genre_dic = t[1]

with open('data/get_ratings.pkl', 'rb') as f:
    lst = pickle.load(f)

In [3]:
def grad_A(Ui, Yij, Vj, a, b, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the updated value of a (by updating by eta times the gradient with respect to a).
    """
    return (1-reg*eta)*a + eta * (Yij - np.dot(Ui,Vj) - a - b)

def grad_B(Ui, Yij, Vj, a, b, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the updated value of a (by updating by eta times the gradient with respect to a).
    """
    return (1-reg*eta)*b + eta * (Yij - np.dot(Ui,Vj) - a - b)

def grad_U(Ui, Yij, Vj, a, b, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    return (1-reg*eta)*Ui + eta * Vj * (Yij - np.dot(Ui,Vj) - a - b)

def grad_V(Vj, Yij, Ui, a, b, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    return (1-reg*eta)*Vj + eta * Ui * (Yij - np.dot(Ui,Vj) - a - b)

def get_err(U, V, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    # Compute mean squared error on each data point in Y; include
    # regularization penalty in error calculations.
    # We first compute the total squared squared error
    err = 0.0
    for (i,j,Yij) in Y:
        err += 0.5 *(Yij - np.dot(U[i-1], V[:,j-1]))**2
    # Add error penalty due to regularization if regularization
    # parameter is nonzero
    if reg != 0:
        U_frobenius_norm = np.linalg.norm(U, ord='fro')
        V_frobenius_norm = np.linalg.norm(V, ord='fro')
        err += 0.5 * reg * (U_frobenius_norm ** 2)
        err += 0.5 * reg * (V_frobenius_norm ** 2)
    # Return the mean of the regularized error
    return err / float(len(Y))

def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    # Initialize U, V, A, B
    U = np.random.random((M,K)) - 0.5
    V = np.random.random((K,N)) - 0.5
    
#     A = np.random.random((M,1)) * 0.01 - 0.005
#     B = np.random.random((1,N)) * 0.01 - 0.005

    A = np.random.random((M,1)) * 0
    B = np.random.random((1,N)) * 0


    size = len(Y)
    delta = None
    indices = np.arange(size)    
    for epoch in range(max_epochs):
        # Run an epoch of SGD
        before_E_in = get_err(U, V, Y, reg)
        np.random.shuffle(indices)
        for ind in indices:
            (i,j, Yij) = Y[ind]
            # Update U[i], V[j]
            U[i-1] = grad_U(U[i-1], Yij, V[:,j-1], A[i-1], B[:,j-1], reg, eta)
            V[:,j-1] = grad_V(V[:,j-1], Yij, U[i-1], A[i-1], B[:,j-1], reg, eta)
            
            A[i-1] = grad_A(U[i-1], Yij, V[:,j-1], A[i-1], B[:,j-1], reg, eta)
            B[:,j-1] = grad_B(V[:,j-1], Yij, U[i-1], A[i-1], B[:,j-1], reg, eta)
            
        # At end of epoch, print E_in
        E_in = get_err(U, V, Y, reg)
        print("Epoch %s, E_in (regularized MSE): %s"%(epoch + 1, E_in))

        # Compute change in E_in for first epoch
        if epoch == 0:
            delta = before_E_in - E_in

        # If E_in doesn't decrease by some fraction <eps>
        # of the initial decrease in E_in, stop early            
        elif before_E_in - E_in < eps * delta:
            break
    return (U, V, get_err(U, V, Y))


In [4]:
num_movies = len(movie_dic)
num_users = 943

In [5]:
m = num_users
n = num_movies
k = 20
eta = 0.1
reg = 0.01

In [6]:
ratings = np.array(lst)

In [7]:
u,v, final_err = train_model(m, n, k, eta, reg, lst, eps=0.0001, max_epochs=300)

Epoch 1, E_in (regularized MSE): 5.26254539991349
Epoch 2, E_in (regularized MSE): 4.512113282723599
Epoch 3, E_in (regularized MSE): 3.84444396818163
Epoch 4, E_in (regularized MSE): 3.377947642769085
Epoch 5, E_in (regularized MSE): 2.942620761094713
Epoch 6, E_in (regularized MSE): 2.6098254998297383
Epoch 7, E_in (regularized MSE): 2.3733078922062947
Epoch 8, E_in (regularized MSE): 2.1381939237819294
Epoch 9, E_in (regularized MSE): 1.9218384132144248
Epoch 10, E_in (regularized MSE): 1.7342440754841482
Epoch 11, E_in (regularized MSE): 1.605809232716293
Epoch 12, E_in (regularized MSE): 1.4709857340575199
Epoch 13, E_in (regularized MSE): 1.4786420269983012


# train/test

In [8]:
Y_train = np.loadtxt('data/train.txt').astype(int)
Y_test = np.loadtxt('data/test.txt').astype(int)

In [9]:
num_movies = len(movie_dic)
num_users = 943
m = num_users
n = num_movies
k = 20
eta = 0.1
reg = 0.1

u,v, final_err = train_model(m, n, k, eta, reg, Y_train, eps=0.0001, max_epochs=300)


Epoch 1, E_in (regularized MSE): 3.5714589727368447
Epoch 2, E_in (regularized MSE): 2.5333840359129285
Epoch 3, E_in (regularized MSE): 2.1174466931852423
Epoch 4, E_in (regularized MSE): 1.9011409562020862
Epoch 5, E_in (regularized MSE): 1.813145853300226
Epoch 6, E_in (regularized MSE): 1.7039875604007362
Epoch 7, E_in (regularized MSE): 1.6809094630137984
Epoch 8, E_in (regularized MSE): 1.6244684722267573
Epoch 9, E_in (regularized MSE): 1.6009022790050338


KeyboardInterrupt: 

In [ ]:
e_in = get_err(u, v, Y_train)
e_out = get_err(u, v, Y_test)
print("e_in is: " + str(e_in))
print("e_out is: " + str(e_out))

# SVD visualization

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=2, n_iter=7, random_state=42)
svd.fit_transform(v)
print("done")

In [ ]:
from numpy.linalg import svd
a, sigma, b = np.linalg.svd(v)

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
np.diag(sigma).shape

In [ ]:
u.shape

In [ ]:
v.shape

In [ ]:
transformer = a[:, :2].T
print(transformer.shape)

In [ ]:
transformed_v = np.matmul(transformer, v).T
print(transformed_v.shape)

In [ ]:
transformed_u = np.matmul(transformer, u.T).T
print(transformed_u.shape)

# visualizations

In [ ]:
indices = np.random.randint(1682, size=10)

In [ ]:
x = [transformed_v[i][0] for i in indices]
y = [transformed_v[i][1] for i in indices]

In [ ]:
plt.scatter(x,y)

In [ ]:
num_ratings_vs_movies = [0 for i in movie_dic.keys()]
for (i, j, yij) in lst:
    num_ratings_vs_movies[j] += 1
num_ratings_vs_movies = np.array(num_ratings_vs_movies)

In [ ]:
indices = num_ratings_vs_movies.argsort()[-10:][::-1]

In [ ]:
x = [transformed_v[i][0] for i in indices]
y = [transformed_v[i][1] for i in indices]
plt.scatter(x,y)

In [ ]:
movie_rating_dic = {id: [] for id in movie_dic.keys()} # dic of movie id and ratings

for rating in lst:
    _, movie_id, r = rating
    movie_rating_dic[movie_id].append(r)

avg_ratings = {movie: sum(movie_rating_dic[movie]) / len(movie_rating_dic[movie]) \
               for movie in movie_rating_dic}

# get ten best movies that received highest average rating
top_10 = sorted(avg_ratings.items(), key=lambda tup: tup[1], reverse=True)[:10]
indices = [t[0] for t in top_10]

In [ ]:
x = [transformed_v[i][0] for i in indices]
y = [transformed_v[i][1] for i in indices]
plt.scatter(x,y)

In [ ]:
fant_movies = movie_genre_dic['Fantasy']
indices = fant_movies[:10]
x = [transformed_v[i][0] for i in indices]
y = [transformed_v[i][1] for i in indices]
plt.scatter(x,y)

In [ ]:
fant_movies = movie_genre_dic['Documentary']
indices = fant_movies[:10]
x = [transformed_v[i][0] for i in indices]
y = [transformed_v[i][1] for i in indices]
plt.scatter(x,y)

In [ ]:
fant_movies = movie_genre_dic["Children's"]
indices = fant_movies[:10]
x = [transformed_v[i][0] for i in indices]
y = [transformed_v[i][1] for i in indices]
plt.scatter(x,y)